# Interpolating ICON-CH2-EPS Temperature Forecasts to Pressure Levels

This notebook demonstrates the full workflow for accessing ICON-CH2-EPS temperature forecasts and performing vertical interpolation from model levels to pressure levels. The data is provided by MeteoSwiss as part of Switzerland’s [Open Government Data (OGD) initiative](https://www.meteoswiss.admin.ch/services-and-publications/service/open-data.html).

The core functionality is powered by the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — developed in-house to simplify working with numerical weather model data. It includes the `ogd_api` module for data access, along with utilities for applying temporal and spatial operators to model output, such as horizontal or vertical interpolation.

The ICON-CH1/2-EPS data is typically provided on terrain-following vertical model levels, which do not correspond directly to atmospheric pressure levels. However, for consistent comparison and analysis, it is often necessary to interpolate forecast data to these pressure levels. The [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library includes the function [interpolate_k2p](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.operators.vertical_interpolation.html#meteodatalab.operators.vertical_interpolation.interpolate_k2p), which interpolates a field from model levels to pressure coordinates.

---

## 🔍 **What You’ll Do in This Notebook**

 🛰️  **Retrieve**  
    Fetch deterministic ICON-CH2-EPS forecast data (e.g., temperature (`T`) and pressure (`P`)) using meteodata-lab's [ogd_api](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html)` module.

 📈  **Interpolate to pressure levels**  
    Interpolate ICON-CH2-EPS forecast data to pressure levels using meteodata-lab's [vertical_interpolation](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.operators.vertical_interpolation.html#) module.

---

## Retrieving Forecasts
In this first part, we retrieve deterministic ICON-CH2-EPS temperature and pressure forecast data. To access this data, we use the [ogd_api](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html) module from the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — a convenient interface for accessing numerical weather forecasts via the [STAC (SpatioTemporal Asset Catalog) API](https://data.geo.admin.ch/api/stac/static/spec/v1/apitransactional.html#tag/Data/operation/getAsset), which provides structured access to Switzerland’s open geospatial data.

#### 📁  Browsing the STAC Catalog (Optional)

If you'd like to explore the ICON-CH1/2-EPS forecast datasets interactively before writing code, you can browse them directly in the STAC catalog:

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH1-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch1?.language=en)

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH2-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch2?.language=en)


Below is a screenshot of the ICON-CH2-EPS collection as seen in the STAC browser interface.


![browser-ch2-resized.PNG](./images/browser-ch2-resized.PNG)

### Creating Requests
To retrieve the forecast data, we first define API requests using the [ogd_api.Request](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html#meteodatalab.ogd_api.Request) class. In this example, we create two requests: one to retrieve the temperature field (`T`), which we aim to interpolate, and another to obtain the pressure field (`P`) on model levels, required for the interpolation.

>⏰ **Forecast Availability**: Forecast data will typically be available a couple of hours after the reference time — due to the model runtime and subsequent upload time. The data remains accessible for 24 hours after upload.

In [ ]:
from datetime import datetime, timezone
from meteodatalab import ogd_api

# Set default date of today midnight in UTC
today_midnight_utc = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)

param_list = ["T", "P"]
req_list = []

for param in param_list:
    req = ogd_api.Request(
        collection="ogd-forecasting-icon-ch2",
        variable=param,
        reference_datetime=today_midnight_utc,
        perturbed=False,
        horizon=f"P0DT0H",
    )
    req_list.append((param,req))


Each argument in the request serves the following purpose:

| Argument             | Description |
|----------------------|-------------|
| `collection`         | Forecast collection to use (e.g., `ogd-forecasting-icon-ch2` for ICON-CH2-EPS). |
| `variable`           | Meteorological variable of interest (`T` = temperature and `P` = pressure). |
| `reference_datetime` | Initialization time of the forecast in **UTC**, provided as either:<br>- [datetime.datetime](https://docs.python.org/3/library/datetime.html#datetime-objects) object (e.g.,<br> &nbsp; `datetime.datetime(2025, 5, 22, 9, 0, 0, tzinfo=datetime.timezone.utc)`) <br>- [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Combined_date_and_time_representations) date string (e.g., `"2025-05-22T09:00:00Z"`)|
| `perturbed`          | If `True`, retrieves ensemble forecast members; if `False`, returns the deterministic forecast. |
| `horizon`            | Forecast lead time, provided as either:<br>– [datetime.timedelta](https://docs.python.org/3/library/datetime.html#timedelta-objects) object (e.g., `datetime.timedelta(hours=0)`) <br>– [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Durations) duration string (e.g., `"P0DT0H"`)|

### Retrieving Data
We now send our list of requests to the API and retrieve the resulting datasets using the [get_from_ogd()](https://meteoswiss.github.io/meteodata-lab/generated/meteodatalab.ogd_api.html#meteodatalab.ogd_api.get_from_ogd) function.
Each response is returned as an **[xarray.DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html)**, which is efficient for handling multi-dimensional data.

> 💡 **Tip**: Use temporary caching with earthkit-data to skip repeated downloads — it's auto-cleaned after the session.
> *For more details, see the [earthkit-data caching docs](https://earthkit-data.readthedocs.io/en/latest/examples/cache.html)*.

> 💡 **Hint**: If you get an error message containing `HTTPError: 403 Client Error: Forbidden for url`, you may be trying to retrieve data older than 24h hours! Please adjust your requests.

In [ ]:
from earthkit.data import config
config.set("cache-policy", "temporary")

da_dict = {}
for param, req in req_list:
    da = ogd_api.get_from_ogd(req)
    da_dict[param]= da

In [ ]:
da_dict["T"]

The resulting `xarray.DataArray` has the following dimensions:

- `eps` (ensemble members): 1 member (for deterministic data)
- `ref_time`: single reference time (e.g. default date of today midnight)
- `lead_time`: single lead time (e.g. +0 hours)
- `z`: vertical levels (e.g. 80 model levels)
- `cell`: 283,876 spatial grid points

It includes attributes specifying some meta data such as:

- `parameter`: information on the meteorogical parameter (e.g. name, units etc.)
- `vcoord_type`: information of the vertical coordinate type, here `model_level`

> &#x1F525; **Expert users:** If you are interested in detailed meta data information, i.e. the GRIB encodings, you can retrieve it from the `metadata` attribute.
Please refer to the [earthkit-data metadata documentation](https://earthkit-data.readthedocs.io/en/latest/examples/metadata.html).

## Data Interpolation

Once the data has been loaded, continue with the interpolation process. The following steps are covered in this section:
- Model level description
- Interpolation from model to pressure levels
    - Methods
    - Comparison
    - Implementation
- Pressure level inspection

### Model level description

As shown in the output above, the parameter `T` includes **80 model levels**. These model levels represent how the atmosphere is divided from the Earth's surface up to the top of the model domain. In the numerical weather model ICON the 80 model levels correspond to the so-called full levels and are numbered top down. They are following the terrain and gradually change into levels of constant height as the distance from the surface increases (see the picture below). For more information about model levels, refer to the [model grid documentation](https://opendatadocs.meteoswiss.ch/e-forecast-data/e2-e3-numerical-weather-forecasting-model#vertical-grid).

<div style="text-align: center;">
  <img src="./images/VerticalLayers.png" width="50%">
</div>

### Interpolation from model to pressure levels
To compare and work with multi-leveled forecast data, it is useful to interpolate to standard pressure levels. The `interpolate_k2p` function included in [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) converts model level (k) to pressure level (p) weather data.


#### Methods
The function `interpolate_k2p` offers three different interpolation approaches.

- linear:

$$ f(x) = f_0 \frac{f_1-f_0}{x_1 -x_0} + f_1 \frac{x-f_0}{x_1 -x_0} $$

- logarithmic:

$$ f(x) = f_0 \frac{(x - x_0)(ln f_1 - ln f_0)}{x_1 -x_0} $$

- nearest surface:

$$ 
f(x) = 
\begin{cases} 
& f_0 \, \text{  if  } \, |x_0 - x| < |x_1 - x|\\
& f_1 \, \text{  if  } \, |x_0 - x| \geq |x_1 - x|
\end{cases}
$$

Where $f(x)$ is the interpolant, $(x_0, f_0)$ and $(x_1, f_1)$ two given data points and $x \in (x_0, x_1)$.

#### Comparison of Interpolation Methods

Each interpolation method produces slightly different results. For example, if you have temperature values at model levels *k* and *k-1* and want to estimate the value at 700 hPa, the result will depend on the method used. The following example illustrates how each of the three methods generates different outputs.

<div style="text-align: center;">
  <img src="./images/interpolation.png" width="50%">
</div>

#### Implementation
The relationship between the gradient of the parameter to be interpolated and the pressure gradient, determines which interpolation methods is most suitable. In this example, we apply logarithmic interpolation to the temperature data. The `interpolate_k2p` function requires the following inputs:
- `field`: parameter to be interpolated
- `mode`: interpolation method ('linear_in_p', 'linear_in_lnp' or 'nearest_sfc')
- `p_field`: predicted pressure values
- `p_tc_values`: target pressure levels
- `p_tc_units`: the unit of the target pressure levels ('Pa' or 'hPa')

In [ ]:
from meteodatalab.operators.vertical_interpolation import interpolate_k2p

target_levels = [500, 550, 600, 650, 700, 750, 800, 850]

T_int = interpolate_k2p(field=da_dict["T"], mode="linear_in_lnp", p_field=da_dict["P"], p_tc_values=target_levels, p_tc_units='hPa')

### Pressure level inspection
As demonstarted below, the interpolated parameter has now the following updated dimensions:
- `z`: 50000 - 85000 Pa (vertical levels)
- `vcoord_type`: pressure (type of vertical coordinates)

In [ ]:
T_int